# 自定义多维列表和多级字典

## 多维列表类

In [4]:
import copy

#创建col列的一维列表
def d1lsit(col = 1, default = None):
    return [default]*col

#创建rol行col列的二维列表
def d2list(rol = 1, col = 1, default = None):
    return [[default]*col for _ in range(rol)]

#创建high层rol行col列的三维列表
def d3list(high = 1, rol = 1, col = 1, default = None):
    ls = [[default]*rol for _ in range(high)]
    for h in range(high):
        for r in range(rol):
            ls[h][r] = [default] * col
    return ls

#创建任意维的列表，维度从高维逐次降维
def mnlist(default = None, *dim):
    dim = list(dim)
    dim.reverse()
    ls = [default]*dim[0]
    for i in range(1,len(dim)):
        ls = [ copy.deepcopy(ls) for _ in range(dim[i]) ]
    return ls

class DoList():
    def __init__(self):
        pass
    #创建col列的一维列表
    def d1list(self, col = 1, default = None):
        return [default]*col

    #创建rol行col列的二维列表
    def d2list(self, rol = 1, col = 1, default = None):
        return [[default]*col for _ in range(rol)]

    #创建high层rol行col列的三维列表
    def d3list(self, high = 1, rol = 1, col = 1, default = None):
        ls = [[default]*rol for _ in range(high)]
        for h in range(high):
            for r in range(rol):
                ls[h][r] = [default] * col
        return ls

    #创建任意维的列表，维度从高维逐次降维
    def mnlist(self, default = None, *dim):
        dim = list(dim)
        dim.reverse()
        ls = [default]*dim[0]
        for i in range(1,len(dim)):
            ls = [ copy.deepcopy(ls) for _ in range(dim[i]) ]
        return ls


In [10]:
#测试多维列表
dolist = DoList()
ls = dolist.d1list(10, None)
ls[5] = -1
print(ls)
ls = dolist.d2list(4, 3, None)
ls[3][2] = -1
print(ls)
ls = dolist.d3list(3,3,3, None)
ls[2][2][2] = -1
print(ls)
ls = dolist.mnlist(None, 3,3,3,3)
ls[2][2][2][2] = -1
print(ls)

ls = dolist.d1list(1, None)
ls[0] = 1
ls.append(10)
print(ls)


[None, None, None, None, None, -1, None, None, None, None]
[[None, None, None], [None, None, None], [None, None, None], [None, None, -1]]
[[[None, None, None], [None, None, None], [None, None, None]], [[None, None, None], [None, None, None], [None, None, None]], [[None, None, None], [None, None, None], [None, None, -1]]]
[[[[None, None, None], [None, None, None], [None, None, None]], [[None, None, None], [None, None, None], [None, None, None]], [[None, None, None], [None, None, None], [None, None, None]]], [[[None, None, None], [None, None, None], [None, None, None]], [[None, None, None], [None, None, None], [None, None, None]], [[None, None, None], [None, None, None], [None, None, None]]], [[[None, None, None], [None, None, None], [None, None, None]], [[None, None, None], [None, None, None], [None, None, None]], [[None, None, None], [None, None, None], [None, None, -1]]]]
[1, 10]


## 多级字典类

In [47]:

#根据键列表创建一级字典
def layer1dict(keys = ['default'], default = None, dt = {}):
    for key in keys:
        dt[key] = default
    return dt

#创建二级字典
def layer2dict(keys1, keys2, default = None, dt = {}):
    for k1 in keys1:
        dt[k1] = {}
    for k2 in keys2:
        dt[k1][k12] = default
    return dt

#创建三级字典
def layer3dict(keys1, keys2, keys3, default = None, dt = {}):
    for k1 in keys1:
        dt[k1] = {}
        for k2 in keys2:
            dt[k1][k2] = {}
            for k3 in keys3:
                dt[k1][k2][k3] = default
    return dt


#创建任意级字典，任意级字典可以看作一颗树
def layersdictdefault(dt={}, keys = [['default']], default = None, curlayer = 0):
    #keys是一个二维列表
    if curlayer == len(keys):
        return dt
    for key in keys[curlayer]:
        dt[key] = {}
        if curlayer == len(keys) - 1:
            dt[key] = default
        layersdictdefault(dt[key], keys, default, curlayer + 1)
    return dt

#任意级字典检查，删除，取值，引用操作的核心
def operatekeyroutecore(dt, keyroute, operator, flag_succ = True, flag_invalid = None, curlayer = 0):
    #operator = 'chek', 'del', 'ref', 'val' is that 'check', 'delete', 'get dict reference of vlaue', 'get value'
    #keyroute为一条键路径
    if len(keyroute) is 0 or len(dt) is 0:
        return flag_invalid
    key = keyroute[curlayer]
    if key not in dt:
        return flag_invalid
    else:
        if curlayer is len(keyroute) - 1:
            if operator is 'chek':
                return falg_succ
            elif operator is 'del':
                del dt[key]
                return flag_succ
            elif operator is 'ref':
                return dt
            elif operator is 'val':
                return dt[key]
            else:
                return flag_invalid
        if type(dt[key]) is not dict:
            return flag_invalid
        return operatekeyroutecore(dt[key], keyroute, operator, flag_succ, flag_invalid, curlayer + 1)

#对核心操作的一次封装
def operatekeyroute(dt = {}, keyroute = ['default'], operator = 'chek', flag_succ = True, flag_invalid = None):
    if type(dt) is not dict or type(keyroute) is not list:
        return flag_invalid
    return operatekeyroutecore(dt, keyroute, operator, flag_succ, flag_invalid, 0)

#使用新值覆盖该键路径下已有的值，如果键路径不存在则创建并给予一个默认值
def updatelayersdictcore(dt = {}, keyroute = ['default'], default = None, curlayer = 0):
    #更新完后根据值的类型，返回值本身的引用或者值所在字典的引用
    if len(keyroute) == 0:
        return None
    key = keyroute[curlayer]
    if key not in dt:
        dt[key] = {}
    if curlayer == len(keyroute) - 1:
        dt[key] = default
        if type(default) is list or type(default) is dict:
            return default
        else:
            return dt
    else:
        if type(dt[key]) != dict:
            dt[key] = {}
        return updatelayersdictcore(dt[key], keyroute, default, curlayer + 1)


#遍历满足keyroute条件的键路径
def traversekeyvaluescore(dt = {}, keyroute = ['*'], values = [], curlayer = 0):
    #keyroute的元素为正则表达式的字符串或者表达所有的*
    if type(dt) is not dict:
        values.append(dt)
        return None
    for key in dt.keys():
        if curlayer < len(keyroute):
            restr = r"%s" % keyroute[curlayer]
            if re.search(restr, key) or keyroute[curlayer] is '*':
                traversekeyvaluescore(dt[key], keyroute, values, curlayer+1)
        else:
            traversekeyvaluescore(dt[key], keyroute, values, curlayer+1)
    return values


#检查键路径是否存在
def checkkeyroute(dt = {}, keyroute = ['default']):
    return operatekeyroute(dt, keyroute, 'chek', True,  False)

#删除该键路径下的值
def dellayersdict(dt = {}, keyroute = ['default']):
    return operatekeyroute(dt, keyroute, 'del', True,  False)

#获取该键路径下的值所在字典的引用
def reflayersdict(dt = {}, keyroute = ['default']):
    return operatekeyroute(dt, keyroute, 'ref', True,  None)

#获取该键路径的值
def getlayersdictvalue(dt = {}, keyroute = ['default']):
    return operatekeyroute(dt, keyroute, 'val', True,  None)

def updatelayersdict(dt = {}, keyroute = ['default'], default = None):
    return updatelayersdictcore(dt, keyroute, default, 0)
   
def traversekeyvalues(dt ={}, keyroute = ['*'], values = []):
    return traversekeyvaluescore(dt, keyroute, values, 0)

#遍历整个字典并将其存储在values列表里
def traversevalues(dt = {}, values = []):
    if type(dt) is not dict:
        values.append(dt)
        return values
    for key in dt.keys():
        traversevalues(dt[key], values)
    return values

    
class DoDict():
    def __init__(self):
        pass
    
    #根据键列表创建一级字典
    def layer1dict(self, keys = ['default'], default = None, dt = {}):
        for key in keys:
            dt[key] = default
        return dt

    #创建二级字典
    def layer2dict(self, keys1, keys2, default = None, dt = {}):
        for k1 in keys1:
            dt[k1] = {}
        for k2 in keys2:
            dt[k1][k12] = default
        return dt

    #创建三级字典
    def layer3dict(self, keys1, keys2, keys3, default = None, dt = {}):
        for k1 in keys1:
            dt[k1] = {}
            for k2 in keys2:
                dt[k1][k2] = {}
                for k3 in keys3:
                    dt[k1][k2][k3] = default
        return dt


    #创建任意级字典，任意级字典可以看作一颗树
    def layersdict(self, keys = [['default']], default = None, curlayer = 0, dt={}):
        #keys是一个二维列表
        if curlayer == len(keys):
            return dt
        for key in keys[curlayer]:
            dt[key] = {}
            if curlayer == len(keys) - 1:
                dt[key] = default
            layersdict(keys, default, curlayer + 1, dt[key])
        return dt

    #任意级字典检查，删除，取值，引用操作的核心
    def __operatekeyroutecore(self, dt, keyroute, operator, flag_succ = True, flag_invalid = None, curlayer = 0):
        #operator = 'chek', 'del', 'ref', 'val' is that 'check', 'delete', 'get dict reference of vlaue', 'get value'
        #keyroute为一条键路径
        if len(keyroute) is 0 or len(dt) is 0:
            return flag_invalid
        key = keyroute[curlayer]
        if key not in dt:
            return flag_invalid
        else:
            if curlayer is len(keyroute) - 1:
                if operator is 'chek':
                    return falg_succ
                elif operator is 'del':
                    del dt[key]
                    return flag_succ
                elif operator is 'ref':
                    return dt
                elif operator is 'val':
                    return dt[key]
                else:
                    return flag_invalid
            if type(dt[key]) is not dict:
                return flag_invalid
            return __operatekeyroutecore(dt[key], keyroute, operator, flag_succ, flag_invalid, curlayer + 1)

    #对核心操作的一次封装
    def operatekeyroute(self, dt = {}, keyroute = ['default'], operator = 'chek', flag_succ = True, flag_invalid = None):
        if type(dt) is not dict or type(keyroute) is not list:
            return flag_invalid
        return self.operatekeyroutecore(dt, keyroute, operator, flag_succ, flag_invalid, 0)

    #使用新值覆盖该键路径下已有的值，如果键路径不存在则创建并给予一个默认值
    def __updatelayersdictcore(self, dt = {}, keyroute = ['default'], default = None, curlayer = 0):
        #更新完后根据值的类型，返回值本身的引用或者值所在字典的引用
        if len(keyroute) == 0:
            return None
        key = keyroute[curlayer]
        if key not in dt:
            dt[key] = {}
        if curlayer == len(keyroute) - 1:
            dt[key] = default
            if type(default) is list or type(default) is dict:
                return default
            else:
                return dt
        else:
            if type(dt[key]) != dict:
                dt[key] = {}
            return __updatelayersdictcore(dt[key], keyroute, default, curlayer + 1)

    #遍历满足keyroute条件的键路径
    def __traversekeyvaluescore(self, dt = {}, keyroute = ['*'], values = [], curlayer = 0):
        #keyroute的元素为正则表达式的字符串或者表达所有的*
        if type(dt) is not dict:
            values.append(dt)
            return None
        for key in dt.keys():
            if curlayer < len(keyroute):
                restr = r"%s" % keyroute[curlayer]
                if re.search(restr, key) or keyroute[curlayer] is '*':
                    __traversekeyvaluescore(dt[key], keyroute, values, curlayer+1)
            else:
                __traversekeyvaluescore(dt[key], keyroute, values, curlayer+1)
        return values

    #检查键路径是否存在
    def checkkeyroute(self, dt = {}, keyroute = ['default']):
        return self.operatekeyroute(dt, keyroute, 'chek', True,  False)

    #删除该键路径下的值
    def dellayersdict(self, dt = {}, keyroute = ['default']):
        return self.operatekeyroute(dt, keyroute, 'del', True,  False)

    #获取该键路径下的值所在字典的引用
    def reflayersdict(self, dt = {}, keyroute = ['default']):
        return self.operatekeyroute(dt, keyroute, 'ref', True,  None)

    #获取该键路径的值
    def getlayersdictvalue(self, dt = {}, keyroute = ['default']):
        return self.operatekeyroute(dt, keyroute, 'val', True,  None)

    def updatelayersdict(self, dt = {}, keyroute = ['default'], default = None):
        return self.updatelayersdictcore(dt, keyroute, default, 0)

    def traversekeyvalues(self, dt ={}, keyroute = ['*'], values = []):
        return self.traversekeyvaluescore(dt, keyroute, values, 0)

    #遍历整个字典并将其存储在values列表里
    def traversevalues(self, dt = {}, values = []):
        if type(dt) is not dict:
            values.append(dt)
            return values
        for key in dt.keys():
            traversevalues(dt[key], values)
        return values

    

In [57]:
#测试多级字典
import re

dodict = DoDict()
dt = dodict.layer1dict(['a','b','c'], None)
dt['a'] = dt['a'] if dt['a'] else []
dt['a'].append('xx')
print(dt)

#一个从数据流中构建字典的示例
def layersdictfromdatas(datas, dt = {}):
    for data in datas:
        keyroute = [data[1],data[2],data[4]]
        val = getlayersdictvalue(dt, keyroute)
        if val is None:
            val = [data[0]]
            #updatelayersdict(dt, keyroute, val)
        else:
            val.append(data[0])
    return dt

{'a': ['xx'], 'b': None, 'c': None}


## 对象转json

In [ ]:
def dicttojsonstr(dt,indent = 4, sort_keys = True):
    return json.dumps(dt, sort_keys = sort_keys, indent =indent)

def dictfromjsonstr(strjson):
    return json.loads(strjson)

def dicttofileinjson(dt,filename = 'dictjson.txt',filepath = '.\\', modle = 'w', indent = 4, sort_keys = True):
    strjson = dicttojsonstr(dt, indent, sort_keys)
    file = filepath+filename
    with open(file,modle) as f:
        f.write(strjson)

def dictfromjsonfile(filename = 'dictjson.txt', filepath = '.\\', modle = 'r'):
    dt = None
    file = filepath+filename
    with open(file,modle) as f:
        dt = dictfromjsonstr(f.read())
    return dt



In [ ]:
#测试对象转成json


In [34]:
import re
s = "[dog|run]"
restr = r"%s" % s
# match = re.search(restr, "dog")
# match = re.search(restr, "runs to cat")
match = re.search(restr, "dog runs to cat")
if match:
    print(res)
else:
    print("no match")
    
if None:
    print("None")
else:
    print("not None")

<_sre.SRE_Match object; span=(0, 1), match='r'>
not None
